In [1]:
import os
import google.generativeai as genai
import math
import requests
import csv
import pandas as pd
import re
import os
import configparser
import json
import unicodedata
import re
import time

Lectura de variables clave

In [2]:
config = configparser.ConfigParser()
config.read("variables.ini")   
geminiKey = config['DEFAULT']['geminiKey']

Modelo Seleccionado (Gemini Pro 1.5)

In [3]:
def limpiezaDocumento(document):
    document = document.replace(" ´", "")
    document = unicodedata.normalize('NFKD', document).encode('ascii', 'ignore').decode('utf-8')
    document = re.sub(r'\s+', ' ', document)  
    return document
    
def entities_LLM(api_key,document, model_name="gemini-1.5-pro", temperature=0, top_p=0.95, top_k=40, max_output_tokens=1024):
    
    
    genai.configure(api_key=api_key)
    
    # Create the model
    generation_config = {
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "max_output_tokens": max_output_tokens,
        "response_mime_type": "text/plain",
    }
    
    model = genai.GenerativeModel(
      model_name=model_name,
      generation_config=generation_config,
    )
    
    chat_session = model.start_chat(
      history=[
        {
          "role": "user",
          "parts": [
              """
            You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph only return that information about artifial intelligence.
            Your task is to identify the entities and relations specified in the user prompt from a given text and produce the output in JSON format.
            This output should be a list of JSON objects, with each object always containing the following keys:
    
            •⁠  ⁠"head": The text of the extracted entity, which must match one of the types specified in the user prompt.
            •⁠  ⁠"head_type": The type of the extracted head entity, selected from the specified list of types.
            •⁠  ⁠"relation": The type of relation between the "head" and the "tail," chosen from the list of allowed relations.
            •⁠  ⁠"tail": The text of the entity representing the tail of the relation.
            •⁠  ⁠"tail_type": The type of the tail entity, also selected from the provided list of types.
    
            Extract as many entities and relationships as possible.
    
            Entity Consistency: Ensure consistency in entity representation. If an entity, like "John Doe," appears multiple times in the text under different names or pronouns (e.g., "Joe," "he"), use always the name. 
    
            Important Notes:
            •⁠  ⁠Do not add any extra explanations or text.
            •⁠  ⁠If no one relation can be optained only return NO
            
    
            allowed_nodes = ["Person", "Organization", "Location", "Method", "ResearchField","Technology","Metric","DataSet","Group","Exprement","Evaluation","Tool","Algoritm"]
            allowed_relations = ["Accuracy","Advantage","Allow","Applied","Analyzed","based_on","Belongs","Better_than","Capable","Compared","Contains","Used","Describe","Developed","Disadvantage","Evaluated","Goal","Implemented","Includes","Interacts","Located","Metric_value","Participants","Produces","Property","Estudied","Uses"]
            
            examples = [
                {
                    "head": "SVM",
                    "head_type": "Method",
                    "relation": "Accuracy",
                    "tail": "98",
                    "tail_type": "Metric",
                },
                {
                    "head": "IA",
                    "head_type": "Technology",
                    "relation": "Used",
                    "tail": "classical automation of control",
                    "tail_type": "ResearchField",
                }
            ]
            Document:
              """
          ],
        },
      ]
    )
    response = chat_session.send_message(document)
    return response   


In [4]:
df = pd.read_csv("dfEntre30y250.csv")
df

In [15]:
## Solo ejecutar si se pierde la linea

cValidos = 0
cFallidos = 0
cTotal = 0
triples = []
pTime = []


In [27]:
## Iteración de bloques de 1000
for x in range(5000,5751):
    try:
        i = df.iloc[x]
        cTotal = cTotal + 1
        
        ## Inicio de tiempo
        start_time = time.time()
        ## Llamado al modelo
        response = entities_LLM(geminiKey,limpiezaDocumento(i.content)) 
        ## Fin de tiempo
        end_time = time.time()
        
        ## Calculo del tiempo con 4 decimales 
        processing_time = round(end_time - start_time, 4)

        ## Guardar con id y tiempo
        pTime.append([x,i.id,processing_time])
        
        ## Fila ejecutada
        print(x,i.id, end=" ")
        
        ## Se transforma la salida a json
        data = json.loads(response.text)
        
        ## Se itera por la salida y se guarda cada elemento 
        for item in data:
            triples.append([i.id,i.eid,i.content,i.vector,i.page_number,i.split_id,i.word_count,item['head'],item['head_type'],item['relation'],item['tail'],item['tail_type']])
        cValidos = cValidos + 1
        print("1")
        time.sleep(1)   
        
    except (json.decoder.JSONDecodeError, TypeError, ValueError, KeyError) as e:
        cFallidos = cFallidos + 1
        print("0")
        


In [28]:
print("Total de documentos con respuesta : ",cValidos)
print("Total de documentos sin respuesta : ",cFallidos)
print("Total de documentos               : ",cTotal)


In [29]:
tiempo = pd.DataFrame(pTime)
tiempo.columns = ['numero', 'id','tiempo']
tiempo = tiempo.drop_duplicates('id')

horas = int(round(tiempo['tiempo'].sum()/60/60,10)) 
minutos = int((round(tiempo['tiempo'].sum()/60/60,10) - horas) * 60)
promedio = round(tiempo['tiempo'].sum()/len(tiempo), 2)

print(f"Tiempo Total                      :{horas} horas y {minutos} minutos")
print(f"Tiempo Promedio                   :{promedio} segundos")

In [30]:
T = pd.DataFrame(triples)
T

In [31]:
T.to_csv("E_T3kE.csv", index=False, encoding="utf-8")